In [17]:
import numpy as np
from data import Vertebral_column
from data import Co_Author
from Processing_Data import Abanole
from Processing_Data import Ecoli
from Processing_Data import Ecloli1
from Processing_Data import Ecoli3
from Processing_Data import Glass1
from Processing_Data import Glass4
from Processing_Data import Haberman
from Processing_Data import Waveform
from Processing_Data import New_thyroid2
from Processing_Data import Page_blocks
from Processing_Data import Pima_Indians_Diabetes
from Processing_Data import Satimage
from Processing_Data import Transfusion
from Processing_Data import Yeast
from data import indian_liver_patient
#from data import spect_heart
from wsvm.application import Wsvm
from svm.application import Svm
from sklearn.svm import SVC
#from sklearn.metrics import f1_score
from sklearn.metrics  import classification_report,precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import _safe_indexing
from sklearn import metrics
import math
from datetime import datetime
from fuzzy.weight import fuzzy

In [18]:
def svm_lib(X_train, y_train,X_test):
    svc=SVC(probability=True, kernel='linear')
    model = svc.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [19]:
def wsvm(C,X_train, y_train,X_test,distribution_weight=None):
    model = Wsvm(C,distribution_weight)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [20]:
def svm(C,X_train, y_train,X_test):
    model = Svm(C)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [21]:
def is_tomek(X,y, class_type):
    nn = NearestNeighbors(n_neighbors=2)
    nn.fit(X)
    nn_index = nn.kneighbors(X, return_distance=False)[:, 1]
    links = np.zeros(len(y), dtype=bool)
    # find which class to not consider
    class_excluded = [c for c in np.unique(y) if c not in class_type]
    X_dangxet = []
    X_tl = []
    # there is a Tomek link between two samples if they are both nearest
    # neighbors of each others.
    for index_sample, target_sample in enumerate(y):
        if target_sample in class_excluded:
            continue
        if y[nn_index[index_sample]] != target_sample:
            if nn_index[nn_index[index_sample]] == index_sample:
                X_tl.append(index_sample)
                X_dangxet.append(nn_index[index_sample])
                links[index_sample] = True

    return links,X_dangxet,X_tl

In [22]:
def Gmean(y_test,y_pred):
    cm_WSVM = metrics.confusion_matrix(y_test, y_pred)
    sensitivity = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    specificity = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(sensitivity*specificity)
    return specificity,sensitivity,gmean

In [23]:
def metr(X_train,y_test,test_pred,sp,se,gmean):
    #se,sp,gmean = Gmean(y_test,test_pred)
    print("So luong samples: ",len(X_train))
    print("\n",classification_report(y_test, test_pred))
    print("SP      : ",sp)
    print("SE      : ",se)
    print("Gmean   : ",gmean)
    print("F1 Score: ",f1_score(y_test, test_pred))
    print("Accuracy: ",accuracy_score(y_test,test_pred))
    print("AUC     : ",roc_auc_score(y_test, test_pred))
    print("Ma tran nham lan: \n",confusion_matrix(y_test, test_pred))
    

In [24]:
def metr_text(f,X_train,y_test,test_pred,sp,se,gmean):
    #se,sp,gmean = Gmean(y_test,test_pred)
    f.write(f"\n\nSo luong samples Tong: {len(X_train)+len(y_test)}")
    f.write(f"\n\nSo luong samples training: {len(X_train)}")
    f.write(f"\nSo luong samples testing: {len(y_test)}\n")
    f.write("\n"+str(classification_report(y_test, test_pred)))
    f.write(f"\nSP      : {sp:0.4f}")
    f.write(f"\nSE      : {se:0.4f}")
    f.write(f"\nGmean   : {gmean:0.4f}")
    f.write(f"\nF1 Score: {f1_score(y_test, test_pred):0.4f}")
    f.write(f"\nAccuracy: {accuracy_score(y_test,test_pred):0.4f}")
    f.write(f"\nAUC     : {roc_auc_score(y_test, test_pred):0.4f}")
    f.write("\n\nMa tran nham lan: \n"+str(confusion_matrix(y_test, test_pred)))

In [25]:
def compute_weight(X, y,name_method ="actual_hyper_lin", name_function = "exp", beta = None,C = None, gamma = None, u = None, sigma = None):
    method = fuzzy.method()
    function = fuzzy.function()
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    try:
        if name_method == "own_class_center": 
            d = method.own_class_center(X, y)
        elif name_method == "estimated_hyper_lin": # actual_hyper_lin, own_class_center
            d = method.estimated_hyper_lin(X, y)
        elif name_method == "own_class_center_opposite":
            d = method.own_class_center_opposite(X, y)
        elif name_method == 'actual_hyper_lin':
            d = method.actual_hyper_lin(X, y,C = C, gamma = gamma)
        elif name_method == 'own_class_center_divided':
            d = method.own_class_center_divided(X, y)
        else:
            print('dont exist method')
        
        if name_function == "lin":
            W = function.lin(d)
        elif name_function == "exp":
            W = function.exp(d, beta)
        elif name_function == "lin_center_own":
            W = function.lin_center_own(d, pos_index,neg_index)
        elif name_function == 'gau':
            W = function.gau(d, u, sigma)
    except Exception as e:
        print('dont exist function')
        print(e)
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    r_pos = 1
    r_neg = len(pos_index)/len(neg_index)
    m = []
    W = np.array(W)
    m = W[pos_index]*r_pos
    m = np.append(m, W[neg_index]*r_neg)
    return m

In [26]:
def fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train_val, y_train_val,namemethod,namefunction):
    if namemethod =="own_class_center_opposite" and namefunction == "exp":
        distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction,beta = beta_center)
        f.write(f"\n\t Beta 'own_class_center_opposite' with exp = {beta_center}\n")
    elif namemethod =="own_class_center" and namefunction == "exp":
        distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
        f.write(f"\n\t Beta 'own_class_center' with exp = {beta_center}\n")
    elif namemethod =="own_class_center_divided" and namefunction == "exp":
        distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
        f.write(f"\n\t Beta 'own_class_center_divided' with exp = {beta_center}\n")
    elif namemethod =="estimated_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
        f.write(f"\n\t Beta 'estimated_hyper_lin' with exp = {beta_estimate}\n")
    elif namemethod =="actual_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction,beta = beta_actual)
        f.write(f"\n\t Beta 'actual_hyper_lin' with exp = {beta_actual}\n")
    else:   
        distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction)
    return distribution_weight

In [27]:
def data_tomelinks(f,C,X_train_val,y_train_val,X_test_val, y_test_val,n_neighbors,clf=None,namemethod=None,namefunction=None):
    links,xdx,xtl = is_tomek(X_train_val,y_train_val,class_type=[-1.0])
    if clf == None:
        if namemethod == None:
            distribution_weight = compute_weight(X_train_val, y_train_val)
        else:
            distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train_val, y_train_val,namemethod,namefunction)
        clf = Wsvm(C,distribution_weight)
        clf.fit(X_train_val, y_train_val)
    else:
        clf.fit(X_train_val, y_train_val)
    y_predict = clf.predict(X_test_val)
    sensitivity,specificity,gmean = Gmean(y_test_val,y_predict)
    nn2 = NearestNeighbors(n_neighbors=n_neighbors)
    nn2.fit(X_train_val)
    y_nn = []
    y_check_pos = np.array(y_train_val <-2)
    for ind,i in enumerate(xdx):
        y_pred = clf.predict([X_train_val[i]])  #
        if y_pred == -1.0:                          
            knn_X = (nn2.kneighbors([X_train_val[i]])[1]).tolist() 
            for j in knn_X[0]:
                y_nn.append(y_train_val[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn
        else:
            y_check_pos[xtl[ind]] = True

    y_nn = np.array(y_nn)
    if len(y_nn)>0:
        y_nn = np.array_split(y_nn, len(y_nn)/n_neighbors)
    y_check_neg = np.array(y_train_val <-2)
    for i in range(0,len(y_nn)):      #
        if 1 not in y_nn[i][1:]:      # Nếu không có nhãn 1 xung quanh X_train[i] bị dự đoán sai => xóa X_train[i]
            y_check_neg[xdx[i]] = True

    y_check = y_check_neg | y_check_pos
    
    sample_indices_ = np.flatnonzero(np.logical_not(y_check)) 
    ytl = _safe_indexing(y_train_val, sample_indices_)
    Xtl = _safe_indexing(X_train_val, sample_indices_)
    return Xtl,ytl,gmean

In [28]:
for n_neighbors in range(5,4,-1):
    print(n_neighbors)

5


In [29]:
def lfb(f,C,dataset,namemethod,namefunction,sample_train_val,T,X_train_val,y_train_val,X_test_val, y_test_val,n_neighbors,thamso1,thamso2): #loop find the best
    gmean = 0
    for i in range(0,T):
        f.write(f"\t\t Vong Lap thu: T = {i+1}")
        f.write(f"\n===================================================================================================================")
        f.write(f"\n\n\tFuzzy SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
        f.write(f"\nSo luong mau training_val ban dau: {sample_train_val}")
        X_train_val, y_train_val, gmean2 = data_tomelinks(f,C,X_train_val,y_train_val,X_test_val, y_test_val,n_neighbors,clf=None,namemethod=namemethod,namefunction=namefunction)
        distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train_val, y_train_val,namemethod,namefunction)
        clf = Wsvm(C,distribution_weight)
        #print(X_train_val)
        #print(y_train_val)
        clf.fit(X_train_val, y_train_val)
        #print(clf.predict([X_test_val[5]]))
        pred2 = clf.predict(X_test_val)    #X_val
        sp,se,gmean = Gmean(y_test_val,pred2) #y_val
        
        #metr(X_train_val,y_test_val,pred2,sp,se,gmean)
        metr_text(f,X_train_val,y_test_val,pred2,sp,se,gmean)
        
        # if ((gmean2 - gmean) <= thamso1) or (gmean2 > thamso2):
        #     f.write("\n_____Gmean_ERROR!!!____\n")
        #     print("\n_____Gmean_ERROR!!!____\n")
        #     return X_train_val, y_train_val
        # else:
        #     gmean = gmean2
        f.write(f"\n===================================================================================================================\n")
    return X_train_val, y_train_val

In [30]:
C = 100
thamso1 = 1
thamso2 = 1
T = 3
n_neighbors = 5
test_size = [0.2,0.3,0.4]
testsize_val = 0.2
# data = [Co_Author, Abanole, Ecoli, Ecloli1, Ecoli3, Glass1, Glass4, Haberman, Waveform, New_thyroid2, Page_blocks,
#             Pima_Indians_Diabetes, Satimage, Transfusion, Yeast]
data = [Haberman]

# !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!!
beta_center, beta_estimate, beta_actual = 0.8, 0.1, 0.8 # !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!!
# !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!

name_method =["own_class_center_opposite","own_class_center","estimated_hyper_lin","actual_hyper_lin"]
#name_method =["own_class_center_divided"]
name_function = ["lin_center_own","exp"]

time = datetime.now().strftime("%d%m%Y_%H%M%S")
filepath = "./text_script"

#svc lib
svc = SVC(probability=True, kernel='linear')
#svm scratch
svm_scr = Svm(C)
#W.svm

for dataset in data:
    filename = (str(dataset).split("\\")[-1]).split(".")[0]
    f = open(f"{filepath}/Data_{filename}_{time}_Detail.txt", "w")
    f.write(f"\nC = {C}, thamso1 = {thamso1}, thamso2 = {thamso2}, T = {T}, n_neighbors = {n_neighbors}  \n")
    f.write(f"\n\n\tUSING DATASET : {filename}\n")

    f2 = open(f"{filepath}/Data_{filename}_{time}_Main.txt", "w")
    f2.write(f"\nC = {C}, thamso1 = {thamso1}, thamso2 = {thamso2}, T = {T}, n_neighbors = {n_neighbors}  \n")
    f2.write(f"\n\n\tUSING DATASET : {filename}\n")
    

    #print(f"\n\tUSING DATASET : {filename}\n")
    for testsize in test_size:
        X_train_val, y_train_val, X_test_val, y_test_val, X_test, y_test = dataset.load_data(test_size=testsize,testsize_val=testsize_val)
        sample = int(len(X_train_val)+len(X_test_val)+len(X_test))
        sample_train = int(sample - (sample*testsize))
        sample_test = int(sample - sample_train)
        sample_train_val = int(sample_train - (sample_train*testsize_val))
        sample_test_val = int(sample_train - sample_train_val)
        #print(f"\t======== TestSize: {testsize} ========")
        #print("So luong sample nguyen ban ban dau: ",len(X_train_val)+len(X_test_val)+len(X_test))

        f.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
        f.write(f"So luong sample nguyen ban ban dau: {sample}")
        f.write(f"\nSo luong sample sau khi su dung test_size: Train: {sample_train}; Test: {sample_test}")
        f.write(f"\nSo luong sample sau khi su dung test_size_val: Train_val: {sample_train_val}; Test_val: {sample_test_val}")
        f.write("\n\n\t====== NOT USING TOMEKLINKS ========== \n")

        f2.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
        f2.write(f"So luong sample nguyen ban ban dau: {sample}")
        f2.write(f"\nSo luong sample sau khi su dung test_size: Train: {sample_train}; Test: {sample_test}")
        f2.write(f"\nSo luong sample sau khi su dung test_size_val: Train_val: {sample_train_val}; Test_val: {sample_test_val}")
        f2.write("\n\n\t====== NOT USING TOMEKLINKS ========== \n")

        #Svm library
        f.write("\n\n\tSVM LIBRARY starting...\n")
        f2.write("\n\n\tSVM LIBRARY starting...\n")
        #print("SVM LIBRARY starting...\n")
        test_pred = svm_lib(X_train_val, y_train_val,X_test)
        sp,se,gmean = Gmean(y_test,test_pred)
        #metr(X_train_val,y_test,test_pred,sp,se,gmean)
        metr_text(f,X_train_val,y_test,test_pred,sp,se,gmean)
        metr_text(f2,X_train_val,y_test,test_pred,sp,se,gmean)
        #Svm scratch
        f.write("\n\n\tSVM starting...\n")
        f2.write("\n\n\tSVM starting...\n")
        print("SVM starting...\n")
        test_pred = svm(C,X_train_val, y_train_val,X_test)
        sp,se,gmean = Gmean(y_test,test_pred)
        #metr(X_train_val,y_test,test_pred,sp,se,gmean)
        metr_text(f,X_train_val,y_test,test_pred,sp,se,gmean)
        metr_text(f2,X_train_val,y_test,test_pred,sp,se,gmean)
        #Wsvm
        f.write("\n\n\tW.SVM starting...\n")
        f2.write("\n\n\tW.SVM starting...\n")
        #print("W.SVM starting...\n")
        N, d = X_train_val.shape
        distribution_weight = np.ones(N)
        test_pred = wsvm(C,X_train_val, y_train_val, X_test, distribution_weight)
        sp,se,gmean = Gmean(y_test,test_pred)
        #metr(X_train_val,y_test,test_pred,sp,se,gmean)
        metr_text(f,X_train_val,y_test,test_pred,sp,se,gmean)
        metr_text(f2,X_train_val,y_test,test_pred,sp,se,gmean)
        #FuzyyWsvm
        
        for namemethod in name_method:
            for namefunction in name_function:
                f.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                f2.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train_val, y_train_val,namemethod,namefunction)
                __ = fuzzy_weight(f2,beta_center, beta_estimate, beta_actual,X_train_val, y_train_val,namemethod,namefunction)
                test_pred = wsvm(C,X_train_val, y_train_val, X_test, distribution_weight)
                sp,se,gmean = Gmean(y_test,test_pred)
                #metr(X_train_val,y_test,test_pred,sp,se,gmean)
                metr_text(f,X_train_val,y_test,test_pred,sp,se,gmean)
                metr_text(f2,X_train_val,y_test,test_pred,sp,se,gmean)
        f.write("\n\n\t========== USING TOMEKLINKS ==========\n")
        f2.write("\n\n\t========== USING TOMEKLINKS ==========\n")


        for n_neighbors in range(5,3,-1):
            #Svm library
            f.write(f"\n=================== K = {n_neighbors}  ========================")
            f.write(f"\n*****************************=================================\n")
            f.write("\n\n\tSVM LIBRARY starting...\n")

            f2.write(f"\n=================== K = {n_neighbors}  ========================")
            f2.write(f"\n*****************************=================================\n")
            f2.write("\n\n\tSVM LIBRARY starting...\n")
            print("SVM LIBRARY starting...\n")

            X_train_tl, y_train_tl,_ = data_tomelinks(f,C,X_train_val,y_train_val,X_test_val, y_test_val, n_neighbors, clf=svc)
            for namemethod in name_method:
                for namefunction in name_function:
                    f.write(f"\n\n\tSVM LIBRARY base, Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    f2.write(f"\n\n\tSVM LIBRARY base, Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    X_train_new, y_train_new = lfb(f,C,dataset,namemethod,namefunction,sample_train_val, T, X_train_tl,y_train_tl,X_test_val, y_test_val, n_neighbors, thamso1,thamso2) 
                    distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train_new, y_train_new,namemethod,namefunction)
                    __ = fuzzy_weight(f2,beta_center, beta_estimate, beta_actual,X_train_new, y_train_new,namemethod,namefunction) #just write to f2,non-using
                    test_pred_tl = wsvm(C,X_train_new, y_train_new, X_test, distribution_weight)
                    test_pred_tl_svmlib = svm_lib(X_train_new, y_train_new, X_test)
                    sp,se,gmean = Gmean(y_test, test_pred_tl)
                    spsvml,sesvml,gmeansvml = Gmean(y_test, test_pred_tl_svmlib)
                    #metr(X_train_new, y_test, test_pred_tl, sp, se, gmean)
                    metr_text(f, X_train_new, y_test, test_pred_tl, sp, se, gmean)
                    metr_text(f2, X_train_new, y_test, test_pred_tl, sp, se, gmean)
                    
                    

            #Svm
            # f.write("\n\n\tSVM starting...\n")
            # f2.write("\n\n\tSVM starting...\n")
            # print("SVM  starting...\n")
            # X_train_tl, y_train_tl,_ = data_tomelinks(C,X_train_val,y_train_val,X_test_val, y_test_val, n_neighbors, clf=svm_scr)
            # X_train_new, y_train_new = lfb(f,C,dataset,namemethod,namefunction,sample_train_val, T, X_train_val,y_train_val,X_test_val, y_test_val, n_neighbors, thamso1, thamso2) 
            # test_pred_tl = svm(C,X_train_new, y_train_new, X_test)
            # sp,se,gmean = Gmean(y_test,test_pred_tl)
            # metr(X_train_new, y_test, test_pred_tl, sp, se, gmean)
            # metr_text(f,X_train_new, y_test, test_pred_tl, sp, se, gmean)
            # metr_text(f2, X_train_new, y_test, test_pred_tl, sp, se, gmean)
            #Wsvm

            f.write("\n\n\tW.SVM starting...\n")
            f2.write("\n\n\tW.SVM starting...\n")
            print("W.SVM starting...\n")
            N, d = X_train_val.shape
            distribution_weight = np.ones(N)        #Dung wsvm co W khoi tao = 1 de tao data tomkelinks
            wsvm_scr = Wsvm(C,distribution_weight)
            X_train_tl, y_train_tl,_ = data_tomelinks(f,C,X_train_val,y_train_val,X_test_val, y_test_val, n_neighbors, clf=wsvm_scr)
            for namemethod in name_method:
                for namefunction in name_function:
                    f.write(f"\n\n\tW.SVM base, Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    f2.write(f"\n\n\tW.SVM base, Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                    print("WSVm")
                    X_train_new, y_train_new = lfb(f,C,dataset,namemethod,namefunction,sample_train_val, T, X_train_tl,y_train_tl,X_test_val, y_test_val, n_neighbors, thamso1, thamso2)
                    distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train_new, y_train_new,namemethod,namefunction)
                    __ = fuzzy_weight(f2,beta_center, beta_estimate, beta_actual,X_train_new, y_train_new,namemethod,namefunction)
                    test_pred_tl = wsvm(C,X_train_new, y_train_new, X_test, distribution_weight)   #khoi tao lai W cho data moi'
                    sp,se,gmean = Gmean(y_test,test_pred_tl)
                    #metr(X_train_new, y_test, test_pred_tl, sp, se, gmean)
                    metr_text(f, X_train_new, y_test, test_pred_tl, sp, se, gmean)
                    metr_text(f2, X_train_new, y_test, test_pred_tl, sp, se, gmean)
            
            #FuzyyWsvm
            for namemethod_base in name_method:
                for namefunction_base in name_function:
                    X_train_tl, y_train_tl,_ = data_tomelinks(f2,C,X_train_val,y_train_val,X_test_val, y_test_val,n_neighbors,clf=None,namemethod=namemethod_base,namefunction=namefunction_base)
                    for namemethod in name_method:
                        for namefunction in name_function:
                            f.write(f"\n\n\tFuzzyBase name_method = '{namemethod_base}',name_function = '{namefunction_base}' \nFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                            f2.write(f"\n\n\tFuzzyBase name_method = '{namemethod_base}',name_function = '{namefunction_base}'\n Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                            print("Fuzzy W.SVM starting...\n")
                            X_train_new, y_train_new = lfb(f,C,dataset,namemethod,namefunction,sample_train_val, T, X_train_tl,y_train_tl,X_test_val, y_test_val, n_neighbors, thamso1, thamso2) 
                            distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train_new, y_train_new,namemethod,namefunction)
                            __ = fuzzy_weight(f2,beta_center, beta_estimate, beta_actual,X_train_new, y_train_new,namemethod,namefunction)
                            test_pred_tl = wsvm(C,X_train_new, y_train_new, X_test, distribution_weight)      #Su dung W.fuzzy
                            sp,se,gmean = Gmean(y_test,test_pred_tl)
                            #metr(X_train_new,y_test,test_pred_tl,sp,se,gmean)
                            metr_text(f,X_train_new,y_test,test_pred_tl,sp,se,gmean)
                            metr_text(f2, X_train_new, y_test, test_pred_tl, sp, se, gmean)

    f.write("\n===================================================================================\n")
    f.close()
    f2.write("\n===================================================================================\n")
    f2.close()

SVM starting...

Fuzzy W.SVM name_method = 'own_class_center_opposite',name_function = 'lin_center_own' starting...



C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metr

Fuzzy W.SVM name_method = 'own_class_center_opposite',name_function = 'exp' starting...

Fuzzy W.SVM name_method = 'own_class_center_divided',name_function = 'lin_center_own' starting...

195
195
Fuzzy W.SVM name_method = 'own_class_center_divided',name_function = 'exp' starting...

195
195
Fuzzy W.SVM name_method = 'own_class_center',name_function = 'lin_center_own' starting...

Fuzzy W.SVM name_method = 'own_class_center',name_function = 'exp' starting...

Fuzzy W.SVM name_method = 'estimated_hyper_lin',name_function = 'lin_center_own' starting...

Fuzzy W.SVM name_method = 'estimated_hyper_lin',name_function = 'exp' starting...

Fuzzy W.SVM name_method = 'actual_hyper_lin',name_function = 'lin_center_own' starting...

Fuzzy W.SVM name_method = 'actual_hyper_lin',name_function = 'exp' starting...

SVM LIBRARY starting...

185
184
184
184
184
184
184
184
185
185
185
185
185
185
185
185
W.SVM starting...

WSVm
WSVm
WSVm
185
184
184
184
184
184
184
184
WSVm
185
185
185
185
185
185
185
1

In [31]:
f.close()
f2.close()